In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
using Printf
using JSON
using CSV
using DataFrames
using ColorSchemes
using LaTeXStrings

In [2]:
include("../src/SailorMoon.jl")   # relative path to main file of module
plotly()

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector}}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240
┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyBase not found in current path.
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
└ @ Plots C:\Users\yujit\.julia\packages\Plots\B5j7d\src\backends.jl:552


Plots.PlotlyBackend()

In [3]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename  = "../run/data/grid_search_Tsit5_0525_EMrotThrust.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_EMrotdir_sb1frame 

# 3body parameter
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)

##################################################


Main.SailorMoon.multishoot_params(0.005, 0.22997051088254342, 2.2997051088254343, 0.05869842280217777, 0.0024510521918983535, 5)

In [4]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end


function interpolatecolor(cs, value, vmin, vmax)
    cs = cgrad(cs)
    color = cs[(value - vmin) / (vmax - vmin)]
    return color
end

interpolatecolor (generic function with 1 method)

In [5]:
# in EM rotating frame (km)
moon     = plot_circle(1740, param3b.mu1*param3b.lstar, 0.0)
earth    = plot_circle(6375, -param3b.mu2*param3b.lstar, 0.0)
moon_soi_outer = plot_circle(66000, param3b.mu1*param3b.lstar, 0.0);

In [6]:
fs = 12;   # omajinai for the plotting
cs = :viridis 

:viridis

In [15]:
if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = DataFrame(CSV.File(filename));


8.59258756924519e-21

In [31]:
row = df[85,:]

sv_mid_cart = [row.x_ra, row.y_ra, row.z_ra, row.xdot_ra, row.ydot_ra, row.zdot_ra]
# change the coordinates into cylindrical (only position)
svm_mid_cyl = vcat(SailorMoon.cart2cylind_only_pos(sv_mid_cart), row.m_ra)

tof_leo2mid = row.dt2
tof_mid2lpo = row.dt1
tof = row.tof
rp = row.rp_kep
ra = row.ra_kep
α = row.alpha
m_rp = row.m_rp

θsf = row.thetasf
ϕ0  = row.phi0

x_lpo  = row.x_lpo
y_lpo  = row.y_lpo
z_lpo  = row.z_lpo
vx_lpo = row.xdot_lpo
vy_lpo = row.ydot_lpo
vz_lpo = row.zdot_lpo

2.1051513577599156e-21

In [32]:
svf = [x_lpo, y_lpo, z_lpo, vx_lpo, vy_lpo, vz_lpo, 1.0]
tspan = [0, -tof]
params = [
    param3b.mu2, param3b.mus, param3b.as, pi - θsf, param3b.oml, param3b.omb, 
    1.0, 0.0, 0.0, 
    paramMulti.mdot, paramMulti.tmax, dir_func, param3b.tstar
]
prob = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2_thrust_bal!, svf, tspan, params)
sol = solve(prob, Tsit5(); reltol=1e-12, abstol=1e-12)

# convert the trajectory (SB1 frame) to EMrot frame

sol_EMrot = zeros(size(sol.u, 1), 7)
for (i, x) in enumerate(sol.u)
    t = sol.t[i]
    θm = (pi - θsf) + param3b.oml * t
    sol_EMrot[i,1:6] = SailorMoon.transform_SunB1_to_EMrot(x[1:6], θm, param3b.oml, param3b.as)
    sol_EMrot[i,1:6] = sol_EMrot[i,1:6] * param3b.lstar
#     println(sol_EMrot_[i,1:6])

    sol_EMrot[i,7]   = x[7]
end

tleo = sol.t[end]
uleo = sol.u[end]
println(uleo)
θmleo = (pi - θsf) + param3b.oml * tleo
uleo_EMrot = SailorMoon.transform_SunB1_to_EMrot(uleo[1:6], θmleo, param3b.oml, param3b.as)
println(uleo_EMrot)


pcart = plot(
    size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false, 
    xlabel=L"x, \times10^6\textrm{km}", 
#     ylabel = "test",
    ylabel=L"y, \times10^6\textrm{km}",
    colorbar_titlefontsize=fs,
    fontsize=fs, xlabelfontsize=fs, ylabelfontsize=fs, xtickfontsize=fs-1, ytickfontsize=fs-1, legendfontsize=fs-1
);

plot!(pcart, sol_EMrot[:,1], sol_EMrot[:,2], c=:blue, lw=1.0)
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer",
    xaxis=(formatter=x->string(round(x / 10^6; digits=2))),
    yaxis=(formatter=y->string(round(y / 10^6; digits=1))),
)


[388.7946880287546, 0.018915489604050182, 6.158078054146536e-23, -1.9332974293749805, -7.944138395115034, 2.5857278299167253e-19, 1.02887939754021]
[-0.014785432053036228, -0.029053790224978802, 6.158078054146536e-23, 8.115703365140657, -0.7247653057274195, 2.5857278299167253e-19]


In [14]:
param3b.tstar * 1 / (24*60*60)

4.348383608673837